## Primer inciso que pide elegir activos

In [44]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import rbf_kernel
import matplotlib.pyplot as plt
import yfinance as yf
from IPython.display import display, Markdown
%matplotlib inline

In [45]:
tickers=['GOOGL','META','AAPL','NFLX','NVDA']

f_inicial='2020-01-01'
f_final='2024-11-24'

closes=yf.download(tickers,f_inicial,f_final)['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [46]:
closes

Ticker,AAPL,GOOGL,META,NFLX,NVDA
Date,,,,,
2020-01-02 00:00:00+00:00,72.796028,68.264961,209.150269,329.809998,5.973123
2020-01-03 00:00:00+00:00,72.088295,67.907845,208.043594,325.899994,5.877516
2020-01-06 00:00:00+00:00,72.662720,69.717865,211.961823,335.829987,5.902164
2020-01-07 00:00:00+00:00,72.320992,69.583206,212.420425,330.750000,5.973621
2020-01-08 00:00:00+00:00,73.484344,70.078476,214.573944,339.260010,5.984825
...,...,...,...,...,...
2024-11-18 00:00:00+00:00,228.020004,175.300003,554.400024,847.049988,140.149994
2024-11-19 00:00:00+00:00,228.279999,178.119995,561.090027,871.320007,147.009995
2024-11-20 00:00:00+00:00,229.000000,175.979996,565.520020,883.849976,145.889999


In [47]:
ret = closes.pct_change().dropna()
mu = ret.mean()
mu # vector mu

Ticker
AAPL     0.001136
GOOGL    0.000925
META     0.001209
NFLX     0.001245
NVDA     0.003152
dtype: float64

In [48]:
sigma = ret.cov()
sigma # matriz de covarianza

Ticker,AAPL,GOOGL,META,NFLX,NVDA
Ticker,,,,,
AAPL,0.000404,0.000269,0.000323,0.000267,0.000419
GOOGL,0.000269,0.000419,0.000375,0.000265,0.000420
META,0.000323,0.000375,0.000810,0.000405,0.000522
NFLX,0.000267,0.000265,0.000405,0.000834,0.000466
NVDA,0.000419,0.000420,0.000522,0.000466,0.001166


In [49]:
sigma_inv = np.linalg.inv(sigma)
pd.DataFrame(sigma_inv) # matriz de covarianza inversa

,0,1,2,3,4
0,5195.388057,-1800.618923,-509.223066,-376.379176,-839.246387
1,-1800.618923,5501.590348,-1306.137891,-151.047561,-689.194705
2,-509.223066,-1306.137891,2435.175400,-462.719451,-252.751672
3,-376.379176,-151.047561,-462.719451,1764.428373,-308.802833
4,-839.246387,-689.194705,-252.751672,-308.802833,1644.397709


In [50]:
ones = np.ones(len(mu))
ones # vector de unos

array([1., 1., 1., 1., 1.])

In [51]:
w = (sigma_inv.dot(ones))/(ones.T.dot(sigma_inv).dot(ones))
w # vector de pesos para portafolio de mínima varianza

array([ 0.53034621,  0.49371906, -0.03037938,  0.14783052, -0.14151641])

In [52]:
w_df = pd.DataFrame(w, index=closes.columns, columns=['w'])
w_df.T # pesos en DataFrame

Ticker,AAPL,GOOGL,META,NFLX,NVDA
w,0.530346,0.493719,-0.030379,0.147831,-0.141516


In [53]:
rp = w.T.dot(mu)
rp # rendimiento esperado del portafolio de mínima varianza diario

0.0007601298755218444

In [54]:
mu

Ticker
AAPL     0.001136
GOOGL    0.000925
META     0.001209
NFLX     0.001245
NVDA     0.003152
dtype: float64

## Segundo inciso que pide elegir activos

In [55]:
U = np.array([mu, ones]).T
U

array([[1.13554120e-03, 1.00000000e+00],
       [9.24837214e-04, 1.00000000e+00],
       [1.20872921e-03, 1.00000000e+00],
       [1.24531797e-03, 1.00000000e+00],
       [3.15218155e-03, 1.00000000e+00]])

In [56]:
M = U.T.dot(sigma_inv).dot(U)
pd.DataFrame(M)

,0,1
0,0.008756,2.393449
1,2.393449,3148.736559


In [57]:
M_inv = np.linalg.inv(M)
pd.DataFrame(M_inv)

,0,1
0,144.165099,-0.109584
1,-0.109584,0.000401


In [58]:
u = np.array([[rp],
              [1]])
u

array([[7.60129876e-04],
       [1.00000000e+00]])

In [59]:
w2 = sigma_inv.dot(U).dot(M_inv).dot(u)
w2

array([[ 0.53034621],
       [ 0.49371906],
       [-0.03037938],
       [ 0.14783052],
       [-0.14151641]])

In [60]:
w2_df = pd.DataFrame(w2, index=closes.columns, columns=['w'])
w2_df.T # pesos en DataFrame

Ticker,AAPL,GOOGL,META,NFLX,NVDA
w,0.530346,0.493719,-0.030379,0.147831,-0.141516
